In [1]:
import os
import pandas as pd
from rich import print as rprint

# Import util functions
from helper import set_openai_key,\
                    test_openai_api,\
                    create_openai_client,\
                    print_pretty,\
                    function_to_schema,\
                    count_tiktoken_length

# Import tools
from helper import generate_tree_string,\
                    get_lines_from_file,\
                    search_code, \
                    get_object_definition

print("Import successful!")

Import successful!


In [2]:
# Define paths
comp_dir = "konwinski-prize"
comp_kaggle_evaluation_dir = os.path.join(comp_dir, "kaggle_evaluation")
comp_kprize_setup_dir = os.path.join(comp_dir, "kprize_setup")

comp_data_zip_path = os.path.join(comp_dir, "data.a_zip")
comp_data_dir = os.path.join(comp_dir, "data")
comp_data_parquet_path = os.path.join(comp_data_dir, "data.parquet")
comp_conda_packages_dir = os.path.join(comp_data_dir, "conda_packages")
comp_pip_packages_dir = os.path.join(comp_data_dir, "pip_packages")
comp_repo_configs_dir = os.path.join(comp_data_dir, "repo_configs")
comp_repos_dir = os.path.join(comp_data_dir, "repos")

In [10]:
idx = 2

kprize_df = pd.read_parquet(comp_data_parquet_path)

row = kprize_df.iloc[idx]
problem_statement = row["problem_statement"]
instance_id = row["instance_id"]
repo_path = os.path.join(comp_repos_dir, f'repo__{instance_id}')

print(repo_path)

konwinski-prize/data/repos/repo__astropy__astropy-17048


In [12]:
print(count_tiktoken_length(problem_statement))

print(problem_statement[:1000],"\n...")

1284
QTable cannot take `dimensionless_unscaled` when creating table from `data`
### Description

Attempting to create an table with a column in units `dimensionless_unscaled` leads to a crash, because a guard against setting a column attribute to `np.ma.masked` or `None` triggers a comparison between `np.ma.masked` and Unit which eventually resolves to a `ZeroDivisionError`.

### Expected behavior

A `QTable` is created. 

### How to Reproduce

```python
from astropy.table import QTable, Column
import numpy as np
data = np.ones((5,2))
tt = QTable(data=data, names=["a","weight"],units={"weight":u.dimensionless_unscaled})
```
Results in 
```
---------------------------------------------------------------------------
ZeroDivisionError                         Traceback (most recent call last)
Cell In[15], line 2
      1 data = np.ones((5,2))
----> 2 tt = QTable(data=data, names=["a","weight"],units={"weight":u.dimensionless_unscaled})
      3 tt

File ~/miniconda3/envs/datapipe- 
...


In [5]:
set_openai_key()

test_openai_api()

client = create_openai_client()

API key set successfully.
Hello! How can I assist you today?


#### Run step by step

In [6]:
model_name = "gpt-4o"

system_prompt = """
### Context
You are a software automation agent responsible for diagnosing and resolving issues in a repository.  
Your task is to configure and utilize the provided tools effectively.  

If the available tools are insufficient, analyze the gap, propose suitable alternatives, and justify their necessity.  

### Metadata
- Repository root directory: {repo_path}
"""


messages = []

In [7]:
tools = [generate_tree_string, generate_tree_string, search_code, get_object_definition]
tool_schemas = [function_to_schema(tool) for tool in tools]
tools_map = {tool.__name__: tool for tool in tools}

**Note:** As of now, OpenAI's GPT models have a 1024-character limit for tool function descriptions, and there isn't a GPT model that allows for a larger description length.

In [8]:
# code for executing the tools returned in the model's response
def handle_tool_calls(tool_calls, messages):
    for tool_call in tool_calls:   
        function = tools_map[tool_call.function.name]
        function_args = json.loads(tool_call.function.arguments)
        result = function(**function_args)
        messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
        
    return messages

In [9]:
prompt = "Generate a patch to resolve the following repository issue:\n{problem_statement}"

prompt = prompt.format(problem_statement=problem_statement)

messages.append({"role":"user","content":prompt})

completion = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "system", "content": system_prompt}] + messages,
            tools=tool_schemas,
        )

print_pretty(completion.dict())

{
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": null,
                "function_call": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "function": {
                            "arguments": "{\"root_directory\":\".\",\"search_string\":\"def _set_column_attribute\"}",
                            "name": "search_code"
                        },
                        "id": "call_fL7LET5jLEukvLmv3xX9nF5D",
                        "type": "function"
                    }
                ]
            }
        }
    ],
    "created": 1740875946,
    "id": "chatcmpl-B6ReEu3IaPnoo22wOMCOJ8QjvCcXS",
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": "fp_eb9d